In [2]:

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import requests
import io
from tqdm import tqdm_notebook

url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"

# The intermediat request() and StringIO() are needed to decode characters like 0xa3, British pound
s = requests.get(url).content
df_london = pd.read_csv(io.StringIO(s.decode("latin-1")))

# Use subset of data for testing
df_london = df_london.sample(250)
#df_london.info()
type(df_london)

pandas.core.frame.DataFrame

In [3]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

In [5]:
from tqdm.notebook import tqdm
from time import sleep

output = widgets.Output()
plot_output = widgets.Output()

dropdown_year = widgets.Dropdown(options =    unique_sorted_values_plus_ALL(df_london.year))
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.purpose))
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1)

def common_filtering(year, purpose, num):
    output.clear_output()
    plot_output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & 
                                  (df_london.purpose == purpose)]

    with output:
        for i in tqdm(range(5), leave=False):
            for j in tqdm(range(15), leave=False):
                sleep(.05)
        display(common_filter.style.applymap(
                    lambda x: colour_ge_value(x, num),
                    subset=['Visits (000s)','Spend (£m)', 'Nights (000s)']))

    with plot_output:
        sns.kdeplot(common_filter['Visits (000s)'], shade=True)
        plt.show()
        

def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)
    
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)

def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)

dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)
display(output)
display(plot_output)


Dropdown(options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

BoundedFloatText(value=5.0, max=100000.0, step=1.0)

Output()

Output()